In [1]:
import numpy as np
from surprise import accuracy

def evaluate_model(ratings, predictions, testset, k=10, threshold=3.5):
    try:
        rmse = accuracy.rmse(predictions, verbose=False)
        mae = accuracy.mae(predictions, verbose=False)
        user_est_true = {}
        for pred in predictions:
            user_id = pred.uid
            if user_id not in user_est_true:
                user_est_true[user_id] = []
            user_est_true[user_id].append((pred.est, pred.r_ui))
        precisions, recalls = [], []
        for user_id, user_ratings in user_est_true.items():
            user_ratings.sort(key=lambda x: x[0], reverse=True)
            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
            n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
            precisions.append(n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0)
            recalls.append(n_rel_and_rec_k / n_rel if n_rel != 0 else 0)
        precision = np.mean(precisions)
        recall = np.mean(recalls)
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        return {'rmse': rmse, 'mae': mae, 'precision': precision, 'recall': recall, 'f1': f1}
    except Exception as e:
        print(f"Error in model evaluation: {str(e)}")
        return {'rmse': np.nan, 'mae': np.nan, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}